In [24]:
import re
import os
from gensim.models import Word2Vec
from sklearn.preprocessing import normalize
from gensim.utils import simple_preprocess
from sklearn.preprocessing import StandardScaler
import joblib

class NLP_Converter:
    def __init__(self, vector_size, window, min_count, workers, sentences):
        # Define the model save path
        # model_save_path = './model/word2vec_model.model'
        # model_dir = os.path.dirname(model_save_path)

        try:
            self.model = Word2Vec.load('model\word2vec_model.model')
        except:
            # # Initialize the Word2Vec model
            # self.model = Word2Vec(
            #     sentences=sentences,
            #     vector_size=vector_size,
            #     window=window,
            #     min_count=min_count,
            #     workers=workers
            # )
            # Save the model to a file
            # self.model.save(model_save_path)
            # print(f"Model saved to {model_save_path}")
            pass

        vocab = set(self.model.wv.index_to_key)
        print(len(vocab))

    def url_to_vector(self,tokens):
        # Get vectors for each token in the URL
        vectors = [self.model.wv[word] for word in tokens if word in self.model.wv]
        # Average the vectors to get a single vector representation for the URL
        if vectors:
            return sum(vectors) / len(vectors)
        else:
            # Return a zero vector if no tokens exist in the Word2Vec vocabulary
            return [0] * self.model.vector_size

    def process_url(self, url):
        """Process a single URL and extract its NLP features."""
        # Tokenize using the same logic as training
        tokens = simple_preprocess(url)
        # print("Tokens in process_url:", tokens)  # Debug

        # Generate vector
        vector = self.url_to_vector(tokens)

        if not any(vector):
            print("Warning: All-zero vector returned for URL")

        # Normalize vector
        # normalized_vector = normalize([vector])[0]

        # return {f'vector_{i+1}': value for i, value in enumerate(normalized_vector)}
        return {f'vector_{i+1}': value for i, value in enumerate(vector)}

In [25]:
import re
from urllib.parse import urlparse, parse_qs
from googlesearch import search

class URLProcessor:
    def __init__(self):
        pass

    def is_valid_url(self, url):
        """Check if the URL is valid."""
        try:
            parsed_url = urlparse(url)
            return parsed_url.scheme in ['http', 'https'] and bool(parsed_url.netloc)
        except Exception:
            return False

    def breakdown_url(self, url):
        try:
            parsed_url = urlparse(url)
            scheme = parsed_url.scheme
            domain = parsed_url.netloc
            path = parsed_url.path
            query = parsed_url.query
            fragment = parsed_url.fragment
            path_parts = path.strip("/").split("/") if path else []
            query_params = parse_qs(query)
            query_keys = list(query_params.keys())
            query_values = [".".join(values) for values in query_params.values()]

            return {
                "original_url": url,
                "scheme": scheme,
                "domain": domain,
                "path": path,
                "path_parts": path_parts,
                "query": query,
                "query_keys": query_keys,
                "query_values": query_values,
                "fragment": fragment,
            }
        except ValueError as e:
            print(f"Error parsing URL {url}: {e}")
            return None

    def having_ip_address(self, url):
        if not url:
            return 0
        match = re.search(
            r"(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\."  # IPv4
            r"([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\."  # IPv4 part
            r"([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\."  # IPv4 part
            r"([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|"  # IPv4 ending
            r"((?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4})",  # IPv6
            url,
        )
        return 1 if match else 0

    def abnormal_url(self, url):
        try:
            hostname = urlparse(url).hostname or ""
            return 1 if hostname in url else 0
        except Exception:
            return 0
    def no_of_embed(self,url):
        urldir = urlparse(url).path
        return urldir.count('//')

    def google_index(self, url):
        try:
            site = search(url, num_results=5)
            return 1 if site else 0
        except Exception:
            return 0

    def count_character(self, url, char):
        if not isinstance(url, str):
            print(f"Invalid URL: {url}")
            return 0
        # print(f"Counting '{char}' in URL: {url}")
        return url.count(char)


    def url_length(self, url):
        return len(url)

    def hostname_length(self, url):
        return len(urlparse(url).netloc)

    def suspicious_words(self, url):
        match = re.search(
            r"PayPal|login|signin|bank|account|update|free|lucky|service|bonus|ebayisapi|webscr",
            url,
        )
        return 1 if match else 0

    def digit_count(self, url):
        return sum(1 for char in url if char.isdigit())

    def letter_count(self, url):
        return sum(1 for char in url if char.isalpha())

    def shortening_service(self, url):
        match = re.search(
            r"bit\\.ly|goo\\.gl|shorte\\.st|go2l\\.ink|x\\.co|ow\\.ly|t\\.co|tinyurl|tr\\.im|is\\.gd|cli\\.gs|"
            r"yfrog\\.com|migre\\.me|ff\\.im|tiny\\.cc|url4\\.eu|twit\\.ac|su\\.pr|twurl\\.nl|snipurl\\.com|"
            r"short\\.to|BudURL\\.com|ping\\.fm|post\\.ly|Just\\.as|bkite\\.com|snipr\\.com|fic\\.kr|loopt\\.us|"
            r"doiop\\.com|short\\.ie|kl\\.am|wp\\.me|rubyurl\\.com|om\\.ly|to\\.ly|bit\\.do|lnkd\\.in|db\\.tt|"
            r"qr\\.ae|adf\\.ly|goo\\.gl|bitly\\.com|cur\\.lv|tinyurl\\.com|ow\\.ly|bit\\.ly|ity\\.im|q\\.gs|is\\.gd|po\\.st|"
            r"bc\\.vc|twitthis\\.com|u\\.to|j\\.mp|buzurl\\.com|cutt\\.us|u\\.bb|yourls\\.org|x\\.co|prettylinkpro\\.com|"
            r"scrnch\\.me|filoops\\.info|vzturl\\.com|qr\\.net|1url\\.com|tweez\\.me|v\\.gd|tr\\.im|link\\.zip\\.net",
            url,
        )
        return 1 if match else 0

    def process_url(self, url):
        if not self.is_valid_url(url):
            return {"error": "Invalid URL"}

        data = self.breakdown_url(url)
        if not data:
            return {"error": "Failed to process URL"}

        # Update features to match the specified order
        data.update({
            "use_of_ip": self.having_ip_address(url),
            "abnormal_url": self.abnormal_url(url),
            "google_index": self.google_index(url),
            "count.": self.count_character(url, "."),
            "count-www": self.count_character(url, "www"),
            "count@": self.count_character(url, "@"),
            "count_dir": self.count_character(urlparse(url).path, "/"),
            "count_embed_domian": self.no_of_embed(url),
            "short_url": self.shortening_service(url),
            "count%": self.count_character(url, "%"),
            "count?": self.count_character(url, "?"),
            "count-": self.count_character(url, "-"),
            "count=": self.count_character(url, "="),
            "url_length": self.url_length(url),
            "hostname_length": self.hostname_length(url),
            "sus_url": self.suspicious_words(url),
            "count-digits": self.digit_count(url),
            "count-letters": self.letter_count(url),
        })
        return data


In [26]:
import pandas as pd

In [27]:
base_df = pd.read_csv("./dataset/v1/base_url.csv")

In [28]:
base_df.shape

(450176, 2)

In [29]:
df = pd.DataFrame()
df['preprocess_url'] = base_df['url'].apply(simple_preprocess)
print("Load the classes")
processor = URLProcessor()
nlp_convert = NLP_Converter(300,5,1,4,df['preprocess_url'])

Load the classes
286432


In [31]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assume processor, nlp_convert, and detect_model are already defined

def feature_extraction(url_df):
    """
    Predict whether URLs in a DataFrame are phishing or not and return the final processed data.

    Args:
        url_df (pd.DataFrame): DataFrame containing a column 'url' with URLs to process.

    Returns:
        pd.DataFrame: DataFrame containing the final processed features for all URLs in the input DataFrame.
    """
    try:
        # Initialize lists to store features and errors
        extracted_features_list = []
        nlp_features_list = []
        errors = []

        for url in url_df['url']:
            try:
                # Preprocess the URL to extract features
                url_features = processor.process_url(url)
                if 'error' in url_features:
                    errors.append({'url': url, 'error': url_features['error']})
                    continue

                # Extract NLP features
                nlp_feature = nlp_convert.process_url(url)

                # Append features for processing
                extracted_features_list.append(url_features)
                nlp_features_list.append(nlp_feature)

            except Exception as url_exception:
                errors.append({'url': url, 'error': str(url_exception)})

        # Check if there are any valid features to process
        if not extracted_features_list or not nlp_features_list:
            return pd.DataFrame(errors)

        # Convert extracted features and NLP features to DataFrames
        feature_columns = [
            'use_of_ip', 'abnormal_url', 'google_index', 'count.', 'count-www',
            'count@', 'count_dir', 'count_embed_domian', 'short_url', 'count%',
            'count?', 'count-', 'count=', 'url_length', 'hostname_length', 'sus_url',
            'count-digits', 'count-letters'
        ]

        features_df = pd.DataFrame(extracted_features_list, columns=feature_columns)
        nlp_features_df = pd.DataFrame(nlp_features_list)

        # Combine the extracted features and NLP features
        combined_df = pd.concat([features_df, nlp_features_df], axis=1)

        # # Normalize the data using StandardScaler
        # scaler = StandardScaler()
        # scaled_data = scaler.fit_transform(combined_df)

        # # Convert to NumPy arrays
        # final_data = scaled_data.astype('float32')

        # # Return the final processed DataFrame
        # final_processed_df = pd.DataFrame(final_data, columns=combined_df.columns)
        combined_df['url'] = url_df['url'].values

        return combined_df

    except Exception as e:
        return pd.DataFrame({'error': [str(e)]})


In [32]:
base_df = base_df.dropna()
base_df.shape

(450176, 2)

In [33]:
base_df['type'] = base_df['type'].map({'phishing': 1, 'legitimate': 0})

In [34]:
base_df.head()

,url,type
0,https://www.google.com,0
1,https://www.youtube.com,0
2,https://www.facebook.com,0
3,https://www.baidu.com,0
4,https://www.wikipedia.org,0


In [36]:
import os
import pandas as pd

def process_in_segments(base_df, segment_count=100, output_dir="segments"):
    """
    Process the input DataFrame in segments, save each segment, and combine the results.

    Args:
        base_df (pd.DataFrame): The base DataFrame to process.
        segment_count (int): The number of segments to divide the DataFrame into.
        output_dir (str): Directory to save intermediate segment files.

    Returns:
        pd.DataFrame: Combined DataFrame of all processed segments.
    """
    os.makedirs(output_dir, exist_ok=True)  # Ensure the output directory exists
    segment_size = len(base_df) // segment_count
    processed_files = []

    for i in range(segment_count):
        try:
            start_idx = i * segment_size
            end_idx = (i + 1) * segment_size if i < segment_count - 1 else len(base_df)
            processed_df = pd.DataFrame()
            segment = pd.DataFrame()
            segment = base_df.iloc[start_idx:end_idx]

            # Check the size of the segment before processing
            print(f"Segment {i} size before processing: {segment.shape}")

            # Skip empty segments
            if segment.empty:
                print(f"Warning: Segment {i} is empty. Skipping.")
                continue
            url_df = segment[['url']]
            # type_df = segment[['type']]
            # Apply feature extraction (ensure feature_extraction is defined elsewhere)
            processed_df = feature_extraction(url_df)


            # Skip empty DataFrames after cleaning
            if processed_df.empty:
                print(f"Warning: Segment {i} has no valid data after cleaning. Skipping.")
                continue

            # save_df = processed_df.join(type_df)
            # Save the segment to a file
            segment_file = os.path.join(output_dir, f"segment_{i}.csv")
            processed_df.to_csv(segment_file, index=False)
            processed_files.append(segment_file)

            print(f"Processed Segment {i + 1} / {segment_count}")

            
        except Exception as e:
            print(f"Error processing segment {i}: {e}")
            continue

    # Combine all segments into a single DataFrame (use try-except for this part too)
    try:
        combined_df = pd.concat([pd.read_csv(file) for file in processed_files], ignore_index=True)
        combined_df = combined_df.join(base_df["type"])
        # Check the size of the combined DataFrame before cleaning
        print(f"Combined DataFrame size before cleaning: {combined_df.shape}")

        # Drop rows with any NaN values in the final DataFrame


        # Check the size of the final cleaned DataFrame
        print(f"Combined DataFrame size after cleaning: {combined_df.shape}")

        # Save the final combined DataFrame
        combined_df.to_csv("final_df_no_norm.csv", index=False)
        print("Final combined DataFrame saved.")
    except Exception as e:
        print(f"Error combining files: {e}")
        combined_df = None

    return combined_df


In [37]:
process_in_segments(base_df)

Segment 0 size before processing: (4501, 2)
Processed Segment 1 / 100
Segment 1 size before processing: (4501, 2)
Processed Segment 2 / 100
Segment 2 size before processing: (4501, 2)
Processed Segment 3 / 100
Segment 3 size before processing: (4501, 2)
Processed Segment 4 / 100
Segment 4 size before processing: (4501, 2)
Processed Segment 5 / 100
Segment 5 size before processing: (4501, 2)
Processed Segment 6 / 100
Segment 6 size before processing: (4501, 2)
Processed Segment 7 / 100
Segment 7 size before processing: (4501, 2)
Processed Segment 8 / 100
Segment 8 size before processing: (4501, 2)
Processed Segment 9 / 100
Segment 9 size before processing: (4501, 2)
Processed Segment 10 / 100
Segment 10 size before processing: (4501, 2)
Processed Segment 11 / 100
Segment 11 size before processing: (4501, 2)
Processed Segment 12 / 100
Segment 12 size before processing: (4501, 2)
Processed Segment 13 / 100
Segment 13 size before processing: (4501, 2)
Processed Segment 14 / 100
Segment 14 

,use_of_ip,abnormal_url,google_index,count.,count-www,count@,count_dir,count_embed_domian,short_url,count%,...,vector_294,vector_295,vector_296,vector_297,vector_298,vector_299,vector_300,url,error,type
0,0.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.897669,0.557779,0.763385,0.165734,-0.631518,0.781222,-0.030970,https://www.google.com,NaN,0
1,0.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.678099,0.432011,0.378192,0.383078,-0.593755,0.776778,-0.604513,https://www.youtube.com,NaN,0
2,0.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.510948,0.617345,0.438434,0.170376,-0.708871,0.919022,-0.545725,https://www.facebook.com,NaN,0
3,0.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.532566,0.311296,0.644168,0.326366,-0.441958,0.480788,-0.122136,https://www.baidu.com,NaN,0
4,0.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.839042,1.002042,0.684355,0.963773,-0.380508,1.413328,-0.263456,https://www.wikipedia.org,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
414095,0.0,1.0,1.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,...,-0.597841,0.170621,0.167211,0.245986,-0.225973,0.633880,-0.213304,https://webmailloutlookupdate.editor.multiscre...,NaN,1
414096,0.0,1.0,1.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,...,-0.854994,0.782215,0.441026,0.160874,-0.429039,0.845653,-0.108863,https://sites.google.com/site/recoeveryhelp2098/,NaN,1
414097,0.0,1.0,1.0,2.0,0.0,0.0,3.0,0.0,0.0,0.0,...,-0.210894,0.235877,-0.015319,0.197519,-0.217140,0.309676,-0.202957,http://kushal308kumar.maskarea.com/fileman/Upl...,NaN,1
414098,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Length of values (4501) does not match length ...,1
